In [ ]:
#구글 드라이브 / COLAB 사용시 
from google.colab import drive

In [ ]:
#구글 드라이브 / COLAB 사용시 
drive.mount('/gdrive', force_remount= True)

Mounted at /gdrive


In [ ]:
#midi 데이터 수정 Music21 라이브러리 설치
!pip install music21

In [ ]:
!nvidia-smi

Thu May 19 07:46:19 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   55C    P8    10W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
#필요한 라이브러리 로드

import glob, pickle
import numpy as np
from music21 import converter, instrument, note, chord, stream
from keras.models import Sequential

from keras.layers import Dense, Dropout, LSTM, CuDNNLSTM

from keras.utils import np_utils

In [ ]:
#set-up
#수정시 , SET-UP 부분만 수정

#LSTM 모델 시퀀스 길이
seq_len = 150

#MIDI 데이터 경로
url = "/gdrive/My Drive/캡스톤 디자인/midi_major_song/장재영"

#결과 midi 파일 이름
output_name = "ex_midi"

#결과 모델 이름 (.h5)
model_name = 'Sample_LSTM_Duration.h5'

#결과 저장 공간 , 모델 저장 공간
output_way = "/gdrive/My Drive"





#set-up 전처리 (수정 x)

model_way = output_way + "/" + model_name
url = url + "/*.mid"
output_way = output_way + "/" + output_name + ".mid"

In [ ]:
#midi_songs에 있는 Note (박자) 정보만 뽑아서 저장할 리스트

notes = []

for i, file in enumerate(glob.glob(url)):
  midi =  converter.parse(file)
  print('\r', 'Parsing file', i, " ",file, end='')

  #Midi file을 notes로 나누어 다루기 위한 변수
  notes_to_parse = None

  #try / except : try 수행 중 에러 발생 시 except 수행

  try :# file has instrument parts
    s2 = instrument.partitionByInstrument(midi)
    notes_to_parse = s2.parts[0].recurse()
  except: # file has notes in a flat structure
    notes_to_parse = midi.flat.notes

  
  for e in notes_to_parse:

    # Note 인 경우 박자(duration) 으로 저장
    if isinstance(e, note.Note):
      notes.append(str(e.duration))

     # Chord 인 경우 각 Note의 박자를 duration 으로 저장
    elif isinstance(e, chord.Chord):
      #코드인 경우 박자처리 => 코드는 박자가 여러개 일 수도 있음
      #그러면 그중 하나만 택하는 걸로 ( 확률상 많은 박자가 선택됨 )
      
      #굳이 노트와 코드로 나누지 않아도 되지만 , 코드 원문을 살리고
      #이후에 어떻게 될지 모르니 분기를 남겨둠
      notes.append(str(e.duration))

 Parsing file 49   /gdrive/My Drive/캡스톤 디자인/midi_major_song/장재영/Beethoven_-_Minuet_in_G_Major_WoO_10_No._2.mid

In [ ]:
# MIDI 파일 정보를 다루기 쉽게 바꿔준다
# n_vocab : 모델 출력의 가짓수를 정하기 위해 박자의 총 가짓수를 센다(박자 길이의 종류)
# set() : 중복되는 원소는 한번만 쓴다 / ex) set("Hello") => {'e', 'H', 'l', 'o'}
n_vocab = (len(set(notes)))

print('Classes of notes : ', n_vocab, '\n')
print('notes : ', notes[:500])
print('length of notes : ', len(notes), '\n')

# pitchnames : notes 배열의 모든 가능한 박자를 정렬해놓은 배열  

pitchnames = sorted(set(item for item in notes))

print('pitchnames : ', pitchnames)
print('length of pitchnames : ', len(pitchnames), '\n')

# create a dictionary to map pitches to integers
# 박자를 정수에 매핑하는 dictionary 자료형 생성

note_to_int = dict((note, number) for number, note in enumerate(pitchnames))

print('note_to_int : ', note_to_int)

Classes of notes :  35 

notes :  ['<music21.duration.Duration 0.5>', '<music21.duration.Duration 0.25>', '<music21.duration.Duration 0.25>', '<music21.duration.Duration 0.5>', '<music21.duration.Duration 0.25>', '<music21.duration.Duration 0.25>', '<music21.duration.Duration 0.25>', '<music21.duration.Duration 0.25>', '<music21.duration.Duration 0.25>', '<music21.duration.Duration 0.0>', '<music21.duration.Duration 0.0>', '<music21.duration.Duration 0.0>', '<music21.duration.Duration 2/3>', '<music21.duration.Duration 0.25>', '<music21.duration.Duration 0.25>', '<music21.duration.Duration 0.25>', '<music21.duration.Duration 0.25>', '<music21.duration.Duration 0.25>', '<music21.duration.Duration 0.25>', '<music21.duration.Duration 0.25>', '<music21.duration.Duration 0.25>', '<music21.duration.Duration 0.25>', '<music21.duration.Duration 0.25>', '<music21.duration.Duration 0.25>', '<music21.duration.Duration 0.25>', '<music21.duration.Duration 0.25>', '<music21.duration.Duration 0.25>',

In [ ]:
#1차 실행 결과 , 어떠한 음이 과하게 존재하여 그 음만 등장함
#예를 들어 8분음표 , 4분 음표가 전체의 95%를 차지 , 클래스 가중치로 이를 해결

#클래스별 가중치 공식 : log2((전체박자수 / 해당음의 박자수)**2)
#ex ) 전체 박자수 : 1000개 / 8분음표 박자수 :100개 
#8분음표의 class 가중치는 약 6.64

#해당 음표가 많을 수록 낮은 가중치 , 적을 수록 높은 가중치를 얻음


import math

ratio = [0 for i in range(len(pitchnames))]
for i in notes:
  ratio[note_to_int[i]] += 1
print(ratio)
print(len(notes))

weight = [0 for i in range(len(pitchnames))]

for i in range(len(pitchnames)):
  print(pitchnames[i],round(100*ratio[i]/len(notes)))
  weight[i] = round(math.log2(((len(notes)/ratio[i]))**2),4)

class_weight = {i : string for i,string in enumerate(weight)}
print(class_weight)

[2561, 9209, 7035, 518, 2446, 28, 349, 14, 1504, 12, 1, 1, 6, 519, 13, 19, 114, 74, 1, 83, 1, 5, 66, 11, 11, 17, 7, 1, 4, 19, 4, 1, 14, 1, 8]
24677
<music21.duration.Duration 0.0> 10
<music21.duration.Duration 0.25> 37
<music21.duration.Duration 0.5> 29
<music21.duration.Duration 0.75> 2
<music21.duration.Duration 1.0> 10
<music21.duration.Duration 1.25> 0
<music21.duration.Duration 1.5> 1
<music21.duration.Duration 1.75> 0
<music21.duration.Duration 1/3> 6
<music21.duration.Duration 10/3> 0
<music21.duration.Duration 13/3> 0
<music21.duration.Duration 16/3> 0
<music21.duration.Duration 17/3> 0
<music21.duration.Duration 2.0> 2
<music21.duration.Duration 2.25> 0
<music21.duration.Duration 2.5> 0
<music21.duration.Duration 2.75> 0
<music21.duration.Duration 2/3> 0
<music21.duration.Duration 20/3> 0
<music21.duration.Duration 3.0> 0
<music21.duration.Duration 3.25> 0
<music21.duration.Duration 3.5> 0
<music21.duration.Duration 3.75> 0
<music21.duration.Duration 4.0> 0
<music21.duration.D

In [ ]:
# LSTM 모델을 위한 Training Dataset 생성

# 박자를 정수로 바꾸어 LSTM 모델의 입출력으로 만들어준다

net_in = []
net_out = []

# LSTM 모델의 입출력을 만들기 위해 ( 전체 길이 - 시퀀스 길이(=100) ) 만큼 반복
# ex) 입력 : 출력 짝지어주기
#     [0 ~ 99] : [100] / [1 ~ 100] : [101] / ... / [전체 길이-100 ~ 전체 길이-1] : [전체 길이]

for i in range(0, len(notes) - seq_len):
  # LSTM 모델 입력과 출력을 만들어준다
  seq_in = notes[i:i + seq_len] # ex) [0:100] => [0 ~ 99]
  seq_out = notes[i + seq_len]  # ex) [100]
  # LSTM은 문자열이 아닌 숫자를 입출력으로 하므로 문자열을 정수로 바꿔야 한다
  net_in.append([note_to_int[char] for char in seq_in]) # 배열 안의 모든 원소에 대해 실행
  net_out.append(note_to_int[seq_out]) # 출력값 하나에 대해 실행
print(np.shape(net_in))
print(np.shape(net_out))

(24527, 150)
(24527,)


In [ ]:
# LSTM 모델 입출력에 맞게 Dataset 전처리

n_patterns = len(net_in)
print('n_patterns : ', n_patterns)

# reshape the input into a format compatible with LSTM layers
# LSTM 입력에 맞는 모양으로 바꿔준다 : (샘플 수, 시퀀스 길이, 자료의 차원)
net_in = np.reshape(net_in, (n_patterns, seq_len, 1))
print('shape of net_in : ', net_in.shape)

# 데이터 범위 정규화 : 0 ~ (n_vocab - 1) => 0 ~ 1
net_in = net_in / float(n_vocab)

# 분류이므로 출력을 One-hot Vector로 만들어주어야 한다.
net_out = np_utils.to_categorical(net_out)
print('shape of net_out : ', net_out.shape)


n_patterns :  24527
shape of net_in :  (24527, 150, 1)
shape of net_out :  (24527, 35)


In [ ]:
# 모델 구성

data_dim = net_in.shape[2]

model = Sequential(name="Sample_LSTM")

# seq_len : 입력으로 넣을 시계열 데이터의 길이 / data_dim : 각 데이터의 차원

model.add(CuDNNLSTM(512, input_shape=(seq_len, data_dim), return_sequences=True))

model.add(Dropout(rate=0.3))

model.add(CuDNNLSTM(512, return_sequences=True))
model.add(Dropout(rate=0.3))

model.add(CuDNNLSTM(512))

model.add(Dense(256))
model.add(Dropout(rate=0.3))

model.add(Dense(n_vocab, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam')

model.summary()


Model: "Sample_LSTM"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 cu_dnnlstm_12 (CuDNNLSTM)   (None, 150, 512)          1054720   
                                                                 
 dropout_12 (Dropout)        (None, 150, 512)          0         
                                                                 
 cu_dnnlstm_13 (CuDNNLSTM)   (None, 150, 512)          2101248   
                                                                 
 dropout_13 (Dropout)        (None, 150, 512)          0         
                                                                 
 cu_dnnlstm_14 (CuDNNLSTM)   (None, 512)               2101248   
                                                                 
 dense_8 (Dense)             (None, 256)               131328    
                                                                 
 dropout_14 (Dropout)        (None, 256)               

In [ ]:
#모델 학습

model.fit(net_in, net_out, epochs=30, batch_size=64, class_weight = class_weight)

Epoch 1/30
384/384 [==============================] - 48s 120ms/step - loss: 11.0823
Epoch 2/30
384/384 [==============================] - 46s 121ms/step - loss: 9.3295
Epoch 3/30
384/384 [==============================] - 47s 122ms/step - loss: 8.5469
Epoch 4/30
384/384 [==============================] - 47s 122ms/step - loss: 8.0014
Epoch 5/30
384/384 [==============================] - 47s 122ms/step - loss: 7.5807
Epoch 6/30
384/384 [==============================] - 47s 122ms/step - loss: 7.2161
Epoch 7/30
384/384 [==============================] - 47s 122ms/step - loss: 6.8262
Epoch 8/30
384/384 [==============================] - 47s 122ms/step - loss: 6.4591
Epoch 9/30
384/384 [==============================] - 47s 121ms/step - loss: 6.0681
Epoch 10/30
384/384 [==============================] - 47s 122ms/step - loss: 5.7044
Epoch 11/30
384/384 [==============================] - 47s 122ms/step - loss: 5.3110
Epoch 12/30
384/384 [==============================] - 47s 122ms/step - l

In [ ]:
# 작곡을 위해 LSTM 모델 입력을 다시 만든다

net_in = []
output = []

for i in range(0, len(notes) - seq_len, 1):
  seq_in = notes[i:i + seq_len]
  seq_out = notes[i + seq_len]

  net_in.append([note_to_int[char] for char in seq_in])
  output.append(note_to_int[seq_out])
  
n_patterns = len(net_in)

In [ ]:
# LSTM 모델이 작곡을 시작하기 위해 시작점으로써 랜덤한 시퀀스를 골라야 한다

# pattern : Dataset의 입력 전체 시퀀스 중 랜덤하게 고른 시퀀스
start = np.random.randint(0, len(net_in)-1)
pattern = net_in[start]
print('Random Sequence : ', pattern)

# int_to_note: 정수를 다시 Note로 바꾸기 위한 dictionary 자료형
int_to_note = dict((number, note) for number, note in enumerate(pitchnames))
print('int_to_note : ', int_to_note)

Random Sequence :  [2, 2, 2, 4, 2, 2, 2, 2, 0, 0, 0, 34, 4, 0, 0, 0, 17, 4, 4, 2, 2, 4, 2, 2, 2, 2, 19, 0, 0, 3, 2, 2, 2, 2, 4, 2, 2, 2, 2, 4, 4, 0, 0, 0, 17, 19, 4, 4, 4, 4, 2, 2, 4, 2, 2, 2, 2, 19, 4, 2, 2, 2, 2, 4, 2, 2, 2, 2, 4, 2, 2, 2, 2, 0, 0, 0, 34, 4, 0, 0, 0, 17, 4, 4, 2, 2, 4, 2, 2, 2, 2, 19, 0, 0, 3, 2, 2, 2, 2, 4, 2, 2, 2, 2, 4, 4, 0, 0, 0, 17, 19, 4, 2, 2, 2, 2, 4, 4, 4, 4, 2, 2, 2, 2, 4, 2, 2, 4, 2, 2, 2, 2, 2, 2, 4, 4, 4, 4, 4, 4, 19, 4, 2, 2, 4, 4, 2, 2, 4, 2]
int_to_note :  {0: '<music21.duration.Duration 0.0>', 1: '<music21.duration.Duration 0.25>', 2: '<music21.duration.Duration 0.5>', 3: '<music21.duration.Duration 0.75>', 4: '<music21.duration.Duration 1.0>', 5: '<music21.duration.Duration 1.25>', 6: '<music21.duration.Duration 1.5>', 7: '<music21.duration.Duration 1.75>', 8: '<music21.duration.Duration 1/3>', 9: '<music21.duration.Duration 10/3>', 10: '<music21.duration.Duration 13/3>', 11: '<music21.duration.Duration 16/3>', 12: '<music21.duration.Duration 17/3>

In [ ]:
# LSTM 모델이 만든 출력값을 저장하기 위한 빈 리스트

pred_out = []

# generate 500 notes
for i in range(0, 500):
  # 랜덤하게 고른 시퀀스를 LSTM 모델 입력에 맞게 바꿔준다
  pred_in = np.reshape(pattern, (1, len(pattern), 1))

  # 입력 범위 정규화 / 0 ~ (n_vocab -1) => 0 ~ 1
  pred_in = pred_in / float(n_vocab)

  # LSTM 모델 사용
  prediction = model.predict(pred_in, verbose=0)

  # 출력 중 값이 가장 큰 Index 선택
  index = np.argmax(prediction)

  # 정수 값을 Note 값으로 변경
  result = int_to_note[index]
  print('\r', 'Predicted ', i, " ",result, end='')
  
  # LSTM이 만든 Note를 하나씩 리스트에 담는다
  pred_out.append(result)
  
  # 다음 입력을 위해 입력에 새 값 추가 후 가장 과거 값 제거
  # ex) [0:99] -> [1:100] -> ... -> [n : n + 99]
  pattern.append(index)
  pattern = pattern[1:len(pattern)]

 Predicted  499   <music21.duration.Duration 1.0>

In [ ]:
print('length of pred_out : ', len(pred_out))
print('pred_out : ', pred_out)

length of pred_out :  500
pred_out :  ['<music21.duration.Duration 0.5>', '<music21.duration.Duration 0.5>', '<music21.duration.Duration 0.5>', '<music21.duration.Duration 0.5>', '<music21.duration.Duration 0.5>', '<music21.duration.Duration 2.0>', '<music21.duration.Duration 0.5>', '<music21.duration.Duration 0.5>', '<music21.duration.Duration 0.5>', '<music21.duration.Duration 0.5>', '<music21.duration.Duration 0.5>', '<music21.duration.Duration 0.5>', '<music21.duration.Duration 1.0>', '<music21.duration.Duration 1.0>', '<music21.duration.Duration 1.0>', '<music21.duration.Duration 0.0>', '<music21.duration.Duration 0.0>', '<music21.duration.Duration 0.0>', '<music21.duration.Duration 0.0>', '<music21.duration.Duration 0.25>', '<music21.duration.Duration 0.25>', '<music21.duration.Duration 1.0>', '<music21.duration.Duration 1.0>', '<music21.duration.Duration 3.0>', '<music21.duration.Duration 1.0>', '<music21.duration.Duration 0.5>', '<music21.duration.Duration 0.5>', '<music21.dura

In [ ]:
# LSTM 모델이 예측한 값들로부터 MIDI 파일을 만들어준다
from music21 import duration
from fractions import Fraction

# MIDI 파일 생성을 위한 빈 리스트
output_notes = []
ori_note = note.Note("C5")
offset = 0
# LSTM 모델 예측 값을 하나씩 처리


for pattern in pred_out:
  new_note = note.Note("C5")

  #str 형태의 예측 duraion 에서 int offset 빼오기
  if pattern.find("/") == -1:
    offset_sum = pattern.replace("<music21.duration.Duration ","").replace(">","").replace(" ","")
    offset_sum = float(offset_sum)
    #print(offset_sum)
  else:
    offset_sum = pattern.replace("<music21.duration.Duration ","").replace(">","").replace(" ","")
    a,b = map(int , offset_sum.split("/"))
    num =  Fraction(a,b)
    offset_sum = num

  offset += offset_sum
  new_note.offset = offset
  new_note.storedInstrument = instrument.Piano()
  output_notes.append(new_note)
  #print(new_note)

# Note/Chord => Stream => MIDI File
midi_stream = stream.Stream(output_notes)
midi_stream.write('midi', fp=output_way)

0.5
1.0
1.5
2.0
2.5
4.5
5.0
5.5
6.0
6.5
7.0
7.5
8.5
9.5
10.5
10.5
10.5
10.5
10.5
10.75
11.0
12.0
13.0
16.0
17.0
17.5
18.0
18.5
19.0
20.0
21.0
22.0
23.0
23.5
24.0
24.5
25.0
26.0
26.5
27.0
28.0
28.5
29.0
29.5
30.0
30.5
31.0
32.0
33.0
34.0
35.0
36.0
37.0
40.0
41.0
41.5
42.0
43.0
44.0
44.5
45.0
46.0
46.5
47.0
47.5
48.0
48.5
49.0
51.0
51.5
52.0
52.5
53.0
53.5
54.0
55.0
56.0
57.0
57.0
57.0
57.0
57.0
57.25
57.5
58.5
59.5
62.5
63.5
64.0
64.5
65.0
65.5
66.5
67.5
68.5
69.5
70.0
70.5
71.0
71.5
72.5
73.0
73.5
74.5
75.0
75.5
76.0
76.5
77.0
77.5
78.5
79.5
80.5
81.5
82.5
83.5
86.5
87.5
88.0
88.5
89.5
90.5
91.0
91.5
92.5
93.0
93.5
94.0
94.5
95.0
95.5
97.5
98.0
98.5
99.0
99.5
100.0
100.5
101.5
102.5
103.5
103.5
103.5
103.5
103.5
103.75
104.0
105.0
106.0
109.0
110.0
110.5
111.0
111.5
112.0
113.0
114.0
115.0
116.0
116.5
117.0
117.5
118.0
119.0
119.5
120.0
121.0
121.5
122.0
122.5
123.0
123.5
124.0
125.0
126.0
127.0
128.0
129.0
130.0
133.0
134.0
134.5
135.0
136.0
137.0
137.5
138.0
139.0
139.5
140.0
140.5
1

'/gdrive/My Drive/duration_test.mid'

In [ ]:
# 모델은 h5 파일 형태로 저장됩니다

model.save(model_way)
# 모델을 불러오기 위해 지워줍니다
del model
# 저장되어 있는 모델을 불러오기 위한 load_model 함수
from keras.models import load_model
# 'model' 에 해당 모델을 불러옵니다
model = load_model(model_way)